In [4]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.ndimage import gaussian_filter

import cfl.util.brain_util as BU
import cfl.util.brain_vis as BV
import cfl.util.fear_mice_functions as fm 

# load response data 
Y = pd.read_pickle('Y.pkl')

behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'

mri_dir, mri_dims, affine, dir_labels = fm.get_global_values()

# load the non-linear mask template
nl_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_nonlinear_mask.nii')
nl_mask = BU.load_brain(nl_mask_path, ori='RPS')
nolin_mask_vec = BU.flatten(nl_mask)

# load all the images in RPS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RPS')

## Examine potential values of sigma (degree of smoothing)

In [32]:
sigmas = [0.5, 1, 2, 3, 4]

one_image = X[20]

dim = (len(sigmas), np.prod(mri_dims))
smoothed_images = np.zeros(dim) 
plot_names = []
for i, sigma in enumerate(sigmas):
    image = BU.unflatten(image, mri_dims)
    smooth_image = gaussian_filter(image, sigma)
    smoothed_images[i] = BU.flatten(smooth_image)
    plot_names.append('Sigma = {}'.format(sigma))

In [34]:
# BV.plot_interactive_panels(smoothed_images[:3], mri_dims, nolin_mask_vec, figsize=(20, 5), dir_labels=dir_labels, std_scale='corr', column_titles=plot_names[:3])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [35]:
# BV.plot_interactive_panels(smoothed_images[3:], mri_dims, nolin_mask_vec, figsize=(20, 5), dir_labels=dir_labels, std_scale='corr', column_titles=plot_names[3:])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [ ]:
# What is the best sigma to choose? I will go with 1 for now 

# Smooth all the data 

In [36]:
# sigma is the std deviation of the Gaussian kernel in each dimension 
# it's 3D because the MRIs are 3D
sigma = (1, 1, 1)

X_smooth = np.zeros(X.shape)
for i, image in enumerate(X): 
    image = BU.unflatten(image, mri_dims)
    smooth_image = gaussian_filter(image, sigma)
    X_smooth[i] = BU.flatten(smooth_image)


In [37]:
np.save('smooth_X_sigma1.npy', X_smooth)

In [ ]:
# To save as Niftis, 
# create a new directory 
# and then save each image in this new folder (with the same names as they had in the original )

## Smooth the Data using the same kernel as in the paper 

In [1]:
# From paper: "aligned images were first smoothed with a full-width half-maximum (FWHM) Gaussian kernel, three times greater than the voxel size (0.3 mm)"

# https://brainder.org/2011/08/20/gaussian-kernels-convert-fwhm-to-sigma/
# ^ this website says that the conversion between FWHM and sigma is 
# FWHM ~=  2.35482004503 * sigma 

In [18]:
# so the sigma to use is 
# sigma = FWHM / conversion_factor 
conversion_factor = 2.35482004503
FWHM = 3 #3 because the FWHM is the size of 3 voxels
paper_sigma = FWHM / conversion_factor
print(paper_sigma)

1.2739827004325424


In [25]:
# sigma is the std deviation of the Gaussian kernel in each dimension 
# it's 3D because the MRIs are 3D
sigma = paper_sigma

X_smooth = np.zeros(X.shape)
for i, image in enumerate(X): 
    image = BU.unflatten(image, mri_dims)
    smooth_image = gaussian_filter(image, sigma)
    X_smooth[i] = BU.flatten(smooth_image)


In [22]:
correct = np.load('smooth_X_paper_sigma.npy')